In [1]:
import os
import warnings

import numpy as np
import pandas as pd
import seaborn as sns

from itertools import combinations
from matplotlib import pyplot as plt
from scipy import stats
from scipy.stats import norm, skew, kurtosis
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error


import xgboost as xgb
import lightgbm as lgb
import catboost as cgb

from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, Ridge

/Users/andrewtal/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
warnings.filterwarnings('ignore')

# Data

In [3]:
def data_convert(x):
    try:
        result = float(str(x).replace(',', '.')) if isinstance(x, np.str) else x
    except:
        result = 0
        
    return result

In [4]:
data_path = '../dataset/biomedia_2020_features_development_set/'

In [5]:
fatty_acids_serum = pd.read_csv(data_path + 'fatty_acids_serum.csv', sep = ';').applymap(data_convert)
fatty_acids_spermatoza = pd.read_csv(data_path + 'fatty_acids_spermatoza.csv', sep = ';').applymap(data_convert)
participant_related_data = pd.read_csv(data_path + 'participant_related_data.csv', sep = ';').applymap(data_convert)
semen_analysis_data = pd.read_csv(data_path + 'semen_analysis_data.csv', sep = ';').applymap(data_convert)
sex_hormones = pd.read_csv(data_path + 'sex_hormones.csv', sep=';').applymap(data_convert)

In [6]:
PD = participant_related_data.drop(['ID'], axis=1)
FASD = fatty_acids_serum.drop(['ID'], axis=1)
FASPD = fatty_acids_spermatoza.drop(['ID'], axis=1)
SEMD = semen_analysis_data.drop(['ID', 'Progressive motility (%)', 
                                     'Non progressive sperm motility (%)',
                                     'Immotile sperm (%)',
                                     'Head defects (%)', 
                                     'Midpiece and neck defects (%)', 
                                     'Tail defects (%)',], axis=1)
SEXD = sex_hormones.drop(['ID'], axis=1)

In [7]:
head_defects = semen_analysis_data['Head defects (%)']; 
midpiece_and_neck_defects = semen_analysis_data['Midpiece and neck defects (%)'];
tail_defects = semen_analysis_data['Tail defects (%)'];
immotile_sperm = semen_analysis_data['Immotile sperm (%)'];
progressive_motility = semen_analysis_data['Progressive motility (%)'];
non_progressive_sperm_motility = semen_analysis_data['Non progressive sperm motility (%)'];

target = {'head_defects': head_defects, 
          'midpiece_and_neck_defects': midpiece_and_neck_defects, 
          'tail_defects': tail_defects, 
          'immotile_sperm': immotile_sperm, 
          'progressive_motility': progressive_motility, 
          'non_progressive_sperm_motility': non_progressive_sperm_motility}

In [8]:
participant_related_data.head(3)

,ID,Abstinence time(days),Body mass index (kg/m²),Age (years)
0,1,4.0,32.5,36
1,2,4.0,33.7,61
2,3,2.0,62.7,51


In [9]:
fatty_acids_serum.head(3)

,ID,Serum C14:0 (myristic acid),Serum C16:0 (palmitic acid),Serum C16:1 (palmitoleic acid),Serum C18:0 (stearic acid),Serum C18:1 n-9 (oleic acid),Serum total C18:1,"Serum C18:2 n-6 (linoleic acid, LA)","Serum C18:3 n-6 (gamma-linoleic acid, GLA)",Serum C20:1 n-9,Serum C20:2 n-6,Serum C20:3 n-6,Serum C20:4 n-6,"Serum C20:5 n-3 (eicosapentaenoic acid, EPA)","Serum C22:5 n-3 (docosapentaenoic acid, DPA)","Serum C22:6 n-3 (docosahexaenoic acid, DHA)"
0,1,0.36,29.72,0.64,13.67,9.00,10.90,19.48,0.30,0.16,0.31,3.37,12.07,2.15,1.35,5.53
1,2,0.28,31.22,0.47,11.84,9.02,11.29,20.34,0.22,0.25,0.26,2.06,11.64,2.80,1.04,6.31
2,3,0.36,27.95,0.47,16.57,8.65,10.36,23.75,0.26,0.19,0.35,3.10,7.37,1.95,0.98,6.34


In [10]:
fatty_acids_spermatoza.head(3)

,ID,Sperm C14:0 (myristic acid),Sperm C15:0 (pentadecanoic acid),Sperm C16:0 (palmitic acid),Sperm C16:1 n-7 (palmitoleic acid),Sperm C17:0,Sperm C18:0 (stearic acid),Sperm C18:1 trans n-6 to n-11,Sperm C18:1 n-9 (oleic acid),Sperm C18:1 n-7 to n-11,...,Sperm C20:1 n-9,Sperm C20:2 n-6,Sperm C22:0,Sperm C20:3 n-6,Sperm C20:4 n-6 and C22:1 n-9 combined,"Sperm C20:5 n-3 (eicosapentaenoic acid, EPA)",Sperm C24:0,Sperm C24:1 n-9,"Sperm C22:5 n-3 (docosapentaenoic acid, DPA)","Sperm C22:6,n3 (docosahexaenoic acid, DHA)"
0,1,1.3,0.2,24.3,0.9,0.3,9.7,0.8,9.8,2.4,...,0.4,0.5,0.1,4.2,2.5,0.06,1.0,1.2,1.3,25.3
1,2,1.5,0.2,26.3,1.3,0.3,8.9,0.9,12.1,2.7,...,0.3,0.9,2.1,2.6,2.6,0.14,1.4,1.6,1.6,17.3
2,3,1.1,0.2,24.8,0.9,0.2,9.0,0.5,12.3,2.3,...,0.4,0.4,1.9,2.2,3.2,0.17,1.7,2.0,1.1,21.5


In [11]:
semen_analysis_data.head(3)

,ID,Sperm concentration (x10⁶/mL),Total sperm count (x10⁶),Ejaculate volume (mL),Sperm vitality (%),Normal spermatozoa (%),Head defects (%),Midpiece and neck defects (%),Tail defects (%),Cytoplasmic droplet (%),Teratozoospermia index,Progressive motility (%),Non progressive sperm motility (%),Immotile sperm (%),"High DNA stainability, HDS (%)","DNA fragmentation index, DFI (%)"
0,1,105.3,363.1,3.5,81,2.0,98.0,11.2,38.0,3.4,1.54,51,19,30,13,29
1,2,45.4,141.6,3.1,50,3.0,97.0,8.5,24.5,2.0,1.36,22,16,62,13,57
2,3,50.0,124.5,2.5,90,6.9,90.6,32.8,14.3,1.5,1.51,18,26,56,11,12


In [12]:
sex_hormones.head(3)

,ID,Seminal plasma anti-Müllerian hormone (AMH) (pmol/L),Serum total testosterone (nmol/L),Serum oestradiol (nmol/L),"Serum sex hormone-binding globulin, SHBG (nmol/L)","Serum follicle-stimulating hormone, FSH (IU/L)","Serum Luteinizing hormone, LH (IU/L)",Serum inhibin B (ng/L),"Serum anti-Müllerian hormone, AMH (pmol/L)"
0,1,64,16.8,0.13,15,2.3,4.2,289,54
1,2,423,15.0,0.15,43,7.2,2.7,164,19
2,3,37,8.2,0.18,32,4.2,4.8,67,47


# 3 fold validation split

In [13]:
test_data_path = '../dataset/biomedia_2020_features_test_set/'

In [14]:
fold1_id_set = [int(item.split('.')[0]) for item in os.listdir(test_data_path + 'features/split_1/')]
fold2_id_set = [int(item.split('.')[0]) for item in os.listdir(test_data_path + 'features/split_2/')]
fold3_id_set = [int(item.split('.')[0]) for item in os.listdir(test_data_path + 'features/split_3/')]

cv_set = [fold1_id_set, fold2_id_set, fold3_id_set]

# Train test split

In [15]:
def train_test_split(x, y, test_set):
    
    all_id_set = set(y.index)
    test_id_set = set(test_set)
    
    train_id_set = all_id_set - test_id_set
    
    x_train = x.iloc[np.array(list(train_id_set))-1, :]
    y_train = y.iloc[np.array(list(train_id_set))-1]

    x_test = x.iloc[np.array(list(test_id_set))-1, :]
    y_test = y.iloc[np.array(list(test_id_set))-1]
    
    return x_train, y_train, x_test, y_test

# Single flod train and pred

In [16]:
def single_model_train_pred(x_train, y_train, x_test, y_test, model, is_pca=True, is_log=True, cutting_method=None):    
    if is_pca:
        scalar = PCA()
        scalar.fit(x_train)
        x_train = scalar.transform(x_train)
        x_test = scalar.transform(x_test)
    
    model.fit(x_train, y_train)
    
    pred = np.expm1(model.predict(x_test)) if is_log else model.predict(x_test)
    
    if cutting_method == 'ceil':
        pred = np.ceil(pred)
    elif cutting_method == 'round':
        pred = np.round(pred)
    elif cutting_method == 'floor':
        pred = np.floor(pred)
    else:
        None
    
    mae = mean_absolute_error(y_test, pred)
    mse = mean_squared_error(y_test, pred)
    rmse = np.sqrt(mse)
    
    return mae, mse, rmse, pred

# Cross validation train and pred

In [17]:
def train_cv(x, y, cv_set, model_name=None, thres=0.75, is_pca=True, is_log=True, cutting_method=None, name='None'):
    
    mae_list = []
    mse_list = []
    rmse_list = []
    pred_list = []
    id_list = []
    
    # Step 1: Train Test Split
    cv_num = len(cv_set)
    
    model = model_dict[model_name]() if model_name != 'xgb' else model_dict[model_name](objective='reg:squarederror')

    for i in range(cv_num):
        x_train, y_train, x_test, y_test = train_test_split(x, y, cv_set[i])
        
        # Step 2: Target Log Transfrom
        if is_log:
            y_train = np.log1p(y_train)
        
        # Step 3: Prediction
        mae, mse, rmse, pred = single_model_train_pred(x_train, y_train, x_test, y_test, model, is_pca, is_log, cutting_method)
        
        pred_list += list(pred)
        id_list += list(np.sort(cv_set[i]))
        
        mae_list.append(mae)
        mse_list.append(mse)
        rmse_list.append(rmse)
        
    _id = np.array(id_list).reshape(-1, 1)
    _pred = np.array(pred_list).reshape(-1, 1)
    
    result = pd.DataFrame(np.concatenate([_id, _pred], axis=1), columns=['ID', name]) 
    result = result.sort_values(['ID'])
    
    return np.mean(mae_list), np.mean(mse_list), np.mean(rmse_list)

# Video feature

In [18]:
def get_video_feature(features_num_per_slide=227, video_feature_extract_func=None):
    feature_list = []

    for i in range(1, 86):
        path_temp = os.path.join(data_path, 'features/' + str(i) + '.csv')
        data_temp = pd.read_csv(path_temp).fillna(0)
        result_temp, feature_num = video_feature_extract_func(data_temp)

        len_temp = len(result_temp)    
        except_dim = feature_num * features_num_per_slide
            
        if len_temp != except_dim:
            result_temp = np.concatenate([result_temp, np.zeros(except_dim-len_temp,)], axis=0)
        assert(len(result_temp) == except_dim)

        feature_list.append(result_temp)

    video_feature_list = np.array(feature_list)
    
    return video_feature_list

# Sub task video feature extract function

In [19]:
def video_feature_extract_non_pro(data):
    
    feature = []
    data_np = np.array(data)
    
    feature.append(np.mean(data_np, axis=1))
    feature.append(np.std(data_np, axis=1))
    feature.append(np.max(data_np, axis=1))
    feature.append(np.min(data_np, axis=1))
    
    feature = np.array(feature)
    feature_num = feature.shape[0]
    feature = feature.reshape(-1)
    return feature, feature_num

In [20]:
def video_feature_extract_pro(data):
    
    feature = []
    data_np = np.array(data)
    
    feature.append(kurtosis(data, axis=1))
    feature.append(skew(data, axis=1))
    feature.append(np.mean(data_np, axis=1))
    
    df = data
    df_even = df.iloc[:,[i%2==0 for i in range(len(df.columns))]]
    df_odd = df.iloc[:,[i%2==1 for i in range(len(df.columns))]]
    
    difference_1 = np.array(df_odd) - np.array(df_even)
    
    feature.append(np.mean(difference_1, axis=1))
    feature.append(np.max(difference_1, axis=1))

    df_even1 = df_odd.iloc[:,[i%2==0 for i in range(len(df_odd.columns))]]
    df_odd1 = df_odd.iloc[:,[i%2==1 for i in range(len(df_odd.columns))]]

    difference_2 = np.array(df_odd1) - np.array(df_even1)

    feature.append(np.max(difference_2, axis=1))
    feature.append(kurtosis(difference_2, axis=1))
    
    df_even2 = df_even1.iloc[:,[i%2==0 for i in range(len(df_even1.columns))]]
    df_odd2 = df_even1.iloc[:,[i%2==1 for i in range(len(df_even1.columns))]]
    
    difference_3 = np.array(df_odd2) - np.array(df_even2)
    feature.append(kurtosis(difference_3, axis=1))
    
    feature = np.array(feature)
        
    feature_num = feature.shape[0]
    feature = feature.reshape(-1)
    return feature, feature_num

In [21]:
def video_feature_extract_is(data):
    
    feature = []
    data_np = np.array(data)
    
    feature.append(kurtosis(data, axis=1))
    feature.append(np.mean(data_np, axis=1))
    feature.append(np.std(data_np, axis=1))
    feature.append(np.min(data_np, axis=1))

    # 1帧之间的变化
    df = data
    df_even = df.iloc[:,[i%2==0 for i in range(len(df.columns))]]
    df_odd = df.iloc[:,[i%2==1 for i in range(len(df.columns))]]
    
    difference_1 = np.array(df_odd) - np.array(df_even)
    
    feature.append(np.mean(difference_1, axis=1))
    feature.append(np.min(difference_1, axis=1))
    feature.append(np.max(difference_1, axis=1))

    df_even1 = df_odd.iloc[:,[i%2==0 for i in range(len(df_odd.columns))]]
    df_odd1 = df_odd.iloc[:,[i%2==1 for i in range(len(df_odd.columns))]]

    difference_2 = np.array(df_odd1) - np.array(df_even1)

    feature.append(np.std(difference_2, axis=1))
    feature.append(np.min(difference_2, axis=1))
    feature.append(np.mean(difference_2, axis=1))
    feature.append(kurtosis(difference_2, axis=1))
    
    df_even2 = df_even1.iloc[:,[i%2==0 for i in range(len(df_even1.columns))]]
    df_odd2 = df_even1.iloc[:,[i%2==1 for i in range(len(df_even1.columns))]]
    
    difference_3 = np.array(df_odd2) - np.array(df_even2)
    
    feature.append(np.mean(difference_3, axis=1))
    feature.append(np.min(difference_3, axis=1))
    
    feature = np.array(feature)
        
    feature_num = feature.shape[0]
    feature = feature.reshape(-1)
    return feature, feature_num

In [22]:
def video_feature_extract_hd(data):
    
    feature = []
    data_np = np.array(data)
    
    feature.append(skew(data, axis=1))
    feature.append(np.max(data_np, axis=1))
    feature.append(np.mean(data_np, axis=1))
    feature.append(np.min(data_np, axis=1))
    
    df = data
    df_even = df.iloc[:,[i%2==0 for i in range(len(df.columns))]]
    df_odd = df.iloc[:,[i%2==1 for i in range(len(df.columns))]]
    
    difference_1 = np.array(df_odd) - np.array(df_even)
    
    feature.append(np.min(difference_1, axis=1))
    feature.append(np.max(difference_1, axis=1))
    feature.append(kurtosis(difference_1, axis=1))

    df_even1 = df_odd.iloc[:,[i%2==0 for i in range(len(df_odd.columns))]]
    df_odd1 = df_odd.iloc[:,[i%2==1 for i in range(len(df_odd.columns))]]

    difference_2 = np.array(df_odd1) - np.array(df_even1)

    feature.append(np.std(difference_2, axis=1))
    feature.append(np.min(difference_2, axis=1))
    feature.append(np.max(difference_2, axis=1))
    
    df_even2 = df_even1.iloc[:,[i%2==0 for i in range(len(df_even1.columns))]]
    df_odd2 = df_even1.iloc[:,[i%2==1 for i in range(len(df_even1.columns))]]
    
    difference_3 = np.array(df_odd2) - np.array(df_even2)
    
    feature.append(np.min(difference_3, axis=1))
    feature.append(np.max(difference_3, axis=1))
    
    feature = np.array(feature)
        
    feature_num = feature.shape[0]
    feature = feature.reshape(-1)
    return feature, feature_num

In [23]:
def video_feature_extract_md(data):
    
    feature = []
    data_np = np.array(data)
    
    feature.append(skew(data, axis=1))
    feature.append(np.max(data_np, axis=1))
    feature.append(np.mean(data_np, axis=1))
    feature.append(np.min(data_np, axis=1))

    df = data
    df_even = df.iloc[:,[i%2==0 for i in range(len(df.columns))]]
    df_odd = df.iloc[:,[i%2==1 for i in range(len(df.columns))]]
    
    difference_1 = np.array(df_odd) - np.array(df_even)
    
    feature.append(np.mean(difference_1, axis=1))
    feature.append(np.min(difference_1, axis=1))
    feature.append(np.max(difference_1, axis=1))
    feature.append(skew(difference_1, axis=1))
    feature.append(np.std(difference_1, axis=1))

    df_even1 = df_odd.iloc[:,[i%2==0 for i in range(len(df_odd.columns))]]
    df_odd1 = df_odd.iloc[:,[i%2==1 for i in range(len(df_odd.columns))]]

    difference_2 = np.array(df_odd1) - np.array(df_even1)

    feature.append(np.std(difference_2, axis=1))
    feature.append(np.min(difference_2, axis=1))
    feature.append(np.max(difference_2, axis=1))
    feature.append(np.mean(difference_2, axis=1))
    feature.append(kurtosis(difference_2, axis=1))

    df_even2 = df_even1.iloc[:,[i%2==0 for i in range(len(df_even1.columns))]]
    df_odd2 = df_even1.iloc[:,[i%2==1 for i in range(len(df_even1.columns))]]
    
    difference_3 = np.array(df_odd2) - np.array(df_even2)
    

    feature.append(skew(difference_3, axis=1))
    feature.append(np.std(difference_3, axis=1))
    feature.append(np.min(difference_3, axis=1))
    feature.append(np.max(difference_3, axis=1))

    feature.append(kurtosis(difference_3, axis=1))
    
    feature = np.array(feature)
        
    feature_num = feature.shape[0]
    feature = feature.reshape(-1)
    return feature, feature_num

In [24]:
def video_feature_extract_td(data):
    
    feature = []
    data_np = np.array(data)
    
    feature.append(skew(data, axis=1))
    feature.append(np.max(data_np, axis=1))
    feature.append(np.mean(data_np, axis=1))
    feature.append(np.std(data_np, axis=1))
    
    df = data
    df_even = df.iloc[:,[i%2==0 for i in range(len(df.columns))]]
    df_odd = df.iloc[:,[i%2==1 for i in range(len(df.columns))]]
    
    difference_1 = np.array(df_odd) - np.array(df_even)
    
    feature.append(np.mean(difference_1, axis=1))
    feature.append(np.max(difference_1, axis=1))
    feature.append(kurtosis(difference_1, axis=1))
    feature.append(skew(difference_1, axis=1))
    feature.append(np.std(difference_1, axis=1))

    df_even1 = df_odd.iloc[:,[i%2==0 for i in range(len(df_odd.columns))]]
    df_odd1 = df_odd.iloc[:,[i%2==1 for i in range(len(df_odd.columns))]]

    difference_2 = np.array(df_odd1) - np.array(df_even1)
    feature.append(np.min(difference_2, axis=1))
    feature.append(np.max(difference_2, axis=1))
    feature.append(skew(difference_2, axis=1))
    
    df_even2 = df_even1.iloc[:,[i%2==0 for i in range(len(df_even1.columns))]]
    df_odd2 = df_even1.iloc[:,[i%2==1 for i in range(len(df_even1.columns))]]
    
    difference_3 = np.array(df_odd2) - np.array(df_even2)
    
    feature.append(skew(difference_3, axis=1))
    feature.append(kurtosis(difference_3, axis=1))
    feature.append(np.mean(difference_3, axis=1))
    feature.append(np.min(difference_3, axis=1))
    
    feature = np.array(feature)
        
    feature_num = feature.shape[0]
    feature = feature.reshape(-1)
    return feature, feature_num

# Results

In [25]:
task_model_dict = {
    'non': 'lasso',
    'pro': 'lasso',
    'imm': 'xgb',
    'hd': 'lgb',
    'md': 'lasso',
    'td': 'lasso'
}

task_feature_dict = {
    'non': video_feature_extract_non_pro,
    'pro': video_feature_extract_pro,
    'imm': video_feature_extract_is,
    'hd': video_feature_extract_hd,
    'md': video_feature_extract_md,
    'td': video_feature_extract_td
}

is_pca_dict = {
    'non': True,
    'pro': True,
    'imm': False,
    'hd': True,
    'md': True,
    'td':  True
}

cutting_method_dict = {
    'non': 'floor',
    'pro': 'round',
    'imm': 'round',
    'hd': 'round',
    'md': 'ceil',
    'td':  'ceil'
}

name_dict = {
    'non': 'non_progressive_sperm_motility',
    'pro': 'progressive_motility',
    'imm': 'immotile_sperm',
    'hd': 'head_defects',
    'md': 'midpiece_and_neck_defects',
    'td':  'tail_defects'
}

data_dict = {
    'PD': PD,
    'FASD': FASD,
    'FASPD': FASPD,
    'SEMD': SEMD,
    'SEXD': SEXD,
}

model_dict = {
    'lasso': Lasso,
    'xgb': xgb.XGBRegressor,
    'lgb': lgb.LGBMRegressor,
}

In [26]:
# for every sub task
for sub_task in task_feature_dict.keys():
    model_name = task_model_dict[sub_task]
    feature_funnc = task_feature_dict[sub_task]
    cutting_method = cutting_method_dict[sub_task]
    task_name = name_dict[sub_task]
    video_feature = pd.DataFrame(get_video_feature(video_feature_extract_func=feature_funnc))
    
    is_log = False
    is_pca = False
    
    print('Methods, MAE, MSE, RMSE')
    print(task_name + '\\\\')
    
    # for every condication(whether or not have PCA and Log, 0: without pca and log, 1:with log, 2:with pca)
    for i in range(3):
        if i == 1:
            is_log = True
            
        if i == 2:
            is_pca = True
    
        x = video_feature
        y = target[task_name]
        
        # only video feature
        mae, mse, rmse = train_cv(x, y, cv_set, model_name,
                                  is_pca=is_pca, is_log=is_log, 
                                  cutting_method=cutting_method, name=task_name)
        
        _name_temp = 'DDF'
        _name_temp = 'DDF+Log' if i == 1 else _name_temp
        _name_temp = 'DDF+Log+PCA' if i == 2 else _name_temp

        print('{0:} & {1:.5f} & {2:.5f} & {3:.5f}\\\\'.format(_name_temp, mae, mse, rmse))
            
        # difference combination of video feature and other data
        for _data_name in data_dict.keys():
            data = data_dict[_data_name]

            x = pd.concat([video_feature, data], axis=1)
            y = target[task_name]

            mae, mse, rmse = train_cv(x, y, cv_set, model_name, 
                                    is_pca=is_pca, is_log=is_log, 
                                    cutting_method=cutting_method, name=task_name)
            
            _name_temp = 'DDF+'+_data_name
            _name_temp = _name_temp + '+Log+PCA' if i == 2 else _name_temp 
            _name_temp = _name_temp + '+Log' if i == 1 else _name_temp
            
            print('{0:} & {1:.5f} & {2:.5f} & {3:.5f}\\\\'.format(_name_temp, mae, mse, rmse))

        # only non_progressive_sperm_motility task run below code
        if (is_log == True) & (is_pca == True) & (task_name =='non_progressive_sperm_motility'):
            x = pd.concat([video_feature, SEMD, FASD], axis=1)
            y = target[task_name]

            mae, mse, rmse = train_cv(x, y, cv_set, model_name, 
                                      is_pca=is_pca, is_log=is_log, 
                                      cutting_method=cutting_method, name=task_name)
            
            _name_temp = 'DDF+SEMD+FASD'
            _name_temp = _name_temp + '+Log' if i == 1 else _name_temp
            _name_temp =  _name_temp + '+Log+PCA' if i == 2 else _name_temp
            
            print('{0:} & {1:.5f} & {2:.5f} & {3:.5f}\\\\'.format(_name_temp, mae, mse, rmse))
    print('\n')

Methods, MAE, MSE, RMSE
non_progressive_sperm_motility\\
DDF & 11.65230 & 307.42323 & 16.97852\\
DDF+PD & 11.91954 & 382.33087 & 18.73772\\
DDF+FASD & 11.65230 & 307.42323 & 16.97852\\
DDF+FASPD & 11.66379 & 307.80255 & 16.98826\\
DDF+SEMD & 12.25616 & 289.08867 & 16.41402\\
DDF+SEXD & 12.20567 & 334.24589 & 17.72679\\
DDF+Log & 7.53284 & 86.88916 & 9.19328\\
DDF+PD+Log & 7.53284 & 86.88916 & 9.19328\\
DDF+FASD+Log & 7.53284 & 86.88916 & 9.19328\\
DDF+FASPD+Log & 7.53284 & 86.88916 & 9.19328\\
DDF+SEMD+Log & 7.33785 & 89.73276 & 9.39156\\
DDF+SEXD+Log & 7.54269 & 86.75452 & 9.17042\\
DDF+Log+PCA & 7.17529 & 81.63013 & 8.92960\\
DDF+PD+Log+PCA & 7.36494 & 84.74507 & 9.12084\\
DDF+FASD+Log+PCA & 7.16338 & 81.26108 & 8.90583\\
DDF+FASPD+Log+PCA & 7.24672 & 82.77299 & 9.00132\\
DDF+SEMD+Log+PCA & 7.00944 & 81.36248 & 8.92113\\
DDF+SEXD+Log+PCA & 7.26765 & 82.85550 & 9.00130\\
DDF+SEMD+FASD+Log+PCA & 6.99754 & 81.37438 & 8.92189\\


Methods, MAE, MSE, RMSE
progressive_motility\\
DDF & 15.74